In [8]:
import re
import urllib.request
import urllib
from bs4 import BeautifulSoup
import json

from multiprocessing import freeze_support
freeze_support()

import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.webdriver import WebDriver
   
import time


class SamoletServiceProcessing:
  
    # def load_url_by_default(self):
    #    self.__urls = [
    #    "https://samolet.ru/api_redesign/flats/?ordering=-order_manual,filter_price_package,pk&free=1"
    #    "https://samolet.ru/api_redesign/flats/?ordering=-order_manual,filter_price_package,pk&free=1&offset=12&limit=12"
    #
    #    "https://samolet.ru/api_redesign/flats/200343/"
    #    ]

    def __generate_jsons(self):
        #for url_base in self.__urls:
        #    next_url = url_base
        #    while next_url:
        #        url = next_url
        #        next_url = ""
        #        print(url)
        #        #sessionid=kjw99ls6bcb4onofwazpcufjmwwa308x
        #        a_request = urllib.request.Request(url)
        #        #a_request.add_header("Cookie", "sessionid=kjw99ls6bcb4onofwazpcufjmwwa308x")
        #        #qrator_jsr=1701810731.684.adBf0Vn0XMh8G9sf-f67qh8uqbbmbh3a42siunqadovrbv0vp-00
        #        a_request.add_header("Cookie", "qrator_jsr=1701810731.684.adBf0Vn0XMh8G9sf-f67qh8uqbbmbh3a42siunqadovrbv0vp-00; Max-Age=300; Path=/;")
        #        with urllib.request.urlopen(a_request) as response:
        #        #with urllib.request.urlopen(url) as response:
        #            self.__page = response.read()
        #            jsons = json.loads(self.__page)
        #            yield (jsons, url)
        #            next_url = jsons.get("next", "")

        
        # формируем настройки для хрома
        options = Options()
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')
#        options.add_argument('--headless')  # отключает открытие браузера
        options.add_argument('--window-size=1280x1696')
        #options.add_argument('user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36')
        options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36') 

        options.add_argument('--disable-extensions')
        options.add_argument('--disable-application-cache')
        options.add_argument('--disable-gpu ')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-setuid-sandbox')
        options.add_argument('--disable-dev-shm-usage')




        
        #options.binary_location = '/usr/bin/chromium-browser'
        options.binary_location = '/usr/bin/chromedriver'

        chrome = uc.Chrome(options=options, headless=True,use_subprocess=False )
#        chrome = uc.Chrome(headless=True,use_subprocess=False )
        #chrome = uc.Chrome()
        
    
        #self.__list_dict = []
        #for url in self.__urls:
        #    for object_params in OnePage(url).list_dict():
        #        self.__list_dict.append(object_params)
        for url_base in self.__urls:
            next_url = url_base
            while next_url:
                url = next_url
                next_url = ""
                print(url)
 
                try:
                    # пробуем получить страницу каталога
                    chrome.get(url)
                    # даем время на загрузку страницы
                    time.sleep(3)
                    # получаем html страницы
                    page = chrome.page_source
                    # варим суп
                    #lenta_soup = BeautifulSoup(page, "lxml")
                    
                    print(page)
                    self.__page = page 
                    jsons = json.loads(self.__page)
                    yield (jsons, url)
                    next_url = jsons.get("next", "")

                except AttributeError as e:
                    print(f"Неудалось получить общий каталог, ошибка - {e}")


                break

    def process(self):
        self.__urls = [
            #"https://samolet.ru/api_redesign/flats/?ordering=-order_manual,filter_price_package,pk&free=1",
            "https://samolet.ru/api_redesign/flats/?ordering=-order_manual,filter_price_package,pk&free=1",
        ]

        self.__list_dict = []
        for jsons, source_url in self.__generate_jsons():
            for e in jsons["results"]:
               
                el = {}
                el[
                    "title"
                ] = f"{e['project']} {e['building']} {str(e['floor_number'])} {str(e['area'])}"
                el["quantity"] = 1
                el["price"] = e["old_price_with_kitchen"]
                el["price_sale"] = e["filter_price"]
                el["datetime_create"] = "1970-01-01T00:00:00.00Z"
                el["brand"] = "samolet.ru"
                el["brand_url"] = "samolet.ru"
                el["category"] = "Новостройки"
                el["url"] = e["url"]
                el["image_url"] = e["plan"]
                el["source_url"] = source_url
                el["apartment_area"] = e["area"]
                el["apartment_floor"] = int(e["floor_number"])
                if e.get("total_floors"):
                    el["apartment_floors_total"] = int(e["total_floors"])
                else:
                    print("not have floors_total in =", source_url)
                    el["apartment_floors_total"] = None
                el["apartment_room"] = int(e["rooms"])
                el["apartment_ppm"] = int(e["ppm"])
                el["apartment_address"] = e["project"]
                #print(e["floor"])
                
                self.__list_dict.append(el)
            print(source_url)

    # def pp(self):
    #    return self.__list_dict

    # def __create_file_name_with_current_datetime(self):
    #    return 'lelev_fresh.json'

    # def save_in_file_with_current_datetime(self):
    #    json_string = json.dumps(self.__list_dict)
    #    file_name = self.__create_file_name_with_current_datetime()
    #    with open(file_name, 'w') as outfile:
    #        json.dump(json_string, outfile)

    def send_in_api(self):
        configuration = openapi_client.Configuration(host="http://absrent.ru:8000")

        with openapi_client.ApiClient(configuration) as api_client:
            api_instance = history_api.HistoryApi(api_client)
            for e in self.__list_dict:
                print(e)
        
            print("Count load object =", len(self.__list_dict))

In [38]:
!ss -plnut

Netid   State    Recv-Q   Send-Q     Local Address:Port      Peer Address:Port  Process                                                                         
udp     UNCONN   0        0          127.0.0.53%lo:53             0.0.0.0:*                                                                                     
udp     UNCONN   0        0                0.0.0.0:631            0.0.0.0:*                                                                                     
tcp     LISTEN   0        100            127.0.0.1:37411          0.0.0.0:*      users:(("python",pid=19264,fd=28))                                             
tcp     LISTEN   0        4096           127.0.0.1:8099           0.0.0.0:*                                                                                     
tcp     LISTEN   0        100            127.0.0.1:48389          0.0.0.0:*      users:(("python",pid=28154,fd=36))                                             
tcp     LISTEN   0        100     

In [9]:
#!pip install undetected_chromedriver

In [10]:
#!pip install undetected-chromedriver

In [11]:
!ls -la /home/jupyter-amtsu/.local/share/undetected_chromedriver/undetected_chromedriver

-rwxr-xr-x 1 jupyter-amtsu jupyter-amtsu 16201816 Feb 11 15:02 /home/jupyter-amtsu/.local/share/undetected_chromedriver/undetected_chromedriver


In [12]:
tssp = SamoletServiceProcessing()
#tssp.load_url_by_default()
tssp.process()
tssp.send_in_api()

WebDriverException: Message: unknown error: cannot connect to chrome at 127.0.0.1:56857
from chrome not reachable
Stacktrace:
#0 0x5642a7365d93 <unknown>
#1 0x5642a7049177 <unknown>
#2 0x5642a703402a <unknown>
#3 0x5642a7082cf8 <unknown>
#4 0x5642a70791cb <unknown>
#5 0x5642a70c3b7c <unknown>
#6 0x5642a70b71e3 <unknown>
#7 0x5642a7087135 <unknown>
#8 0x5642a708813e <unknown>
#9 0x5642a7329e4b <unknown>
#10 0x5642a732ddfa <unknown>
#11 0x5642a73166d5 <unknown>
#12 0x5642a732ea6f <unknown>
#13 0x5642a72fa69f <unknown>
#14 0x5642a7353098 <unknown>
#15 0x5642a7353262 <unknown>
#16 0x5642a7364f34 <unknown>
#17 0x7f93ca2ff609 start_thread


In [14]:
options = uc.ChromeOptions()
options.add_argument('--incognito')
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")

#added new options since issues in Jan 2023 #743
options.add_argument("--disable-extensions")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
#options.add_argument("--headless") #can enable or disable headless, up to your needs

driver = uc.Chrome(use_subprocess=True,options = options)

with driver:
    driver.get('https://www.101katok.ru')

WebDriverException: Message: unknown error: cannot connect to chrome at 127.0.0.1:53131
from chrome not reachable
Stacktrace:
#0 0x55ac99a97d93 <unknown>
#1 0x55ac9977b177 <unknown>
#2 0x55ac9976602a <unknown>
#3 0x55ac997b4cf8 <unknown>
#4 0x55ac997ab1cb <unknown>
#5 0x55ac997f5b7c <unknown>
#6 0x55ac997e91e3 <unknown>
#7 0x55ac997b9135 <unknown>
#8 0x55ac997ba13e <unknown>
#9 0x55ac99a5be4b <unknown>
#10 0x55ac99a5fdfa <unknown>
#11 0x55ac99a486d5 <unknown>
#12 0x55ac99a60a6f <unknown>
#13 0x55ac99a2c69f <unknown>
#14 0x55ac99a85098 <unknown>
#15 0x55ac99a85262 <unknown>
#16 0x55ac99a96f34 <unknown>
#17 0x7efd426f4609 start_thread


In [16]:
driver = uc.Chrome()
driver.get('http://101katok.ru')

WebDriverException: Message: unknown error: cannot connect to chrome at 127.0.0.1:42019
from chrome not reachable
Stacktrace:
#0 0x5619c952dd93 <unknown>
#1 0x5619c9211177 <unknown>
#2 0x5619c91fc02a <unknown>
#3 0x5619c924acf8 <unknown>
#4 0x5619c92411cb <unknown>
#5 0x5619c928bb7c <unknown>
#6 0x5619c927f1e3 <unknown>
#7 0x5619c924f135 <unknown>
#8 0x5619c925013e <unknown>
#9 0x5619c94f1e4b <unknown>
#10 0x5619c94f5dfa <unknown>
#11 0x5619c94de6d5 <unknown>
#12 0x5619c94f6a6f <unknown>
#13 0x5619c94c269f <unknown>
#14 0x5619c951b098 <unknown>
#15 0x5619c951b262 <unknown>
#16 0x5619c952cf34 <unknown>
#17 0x7f2e928bf609 start_thread


In [ ]:
options = uc.ChromeOptions()
driver = uc.Chrome(options = options)
driver.get('http://101katok.ru')

In [ ]:
options = uc.ChromeOptions()
options.add_argument('--incognito')
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")

#added new options since issues in Jan 2023 #743
options.add_argument("--disable-extensions")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
#options.add_argument("--headless") #can enable or disable headless, up to your needs

driver = uc.Chrome(use_subprocess=True,options = options)

driver.get('http://101katok.ru')

In [ ]:
options = uc.ChromeOptions()
options.add_argument('--incognito')
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")

#added new options since issues in Jan 2023 #743
options.add_argument("--disable-extensions")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
#options.add_argument("--headless") #can enable or disable headless, up to your needs

driver = uc.Chrome(use_subprocess=True,options = options)

with driver:
    driver.get('https://101katok.ru')

In [36]:


options = uc.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--window-size=1920,1080")

# get ua from config file
#ua = get_config_by_key("UA", "user_agent")

# update user-agent
#chrome_options.add_argument(f"--user-agent={ua}")
options.add_argument("--incognito")

options.add_argument("--disable-extensions")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--remote-debugging-port=9222")
options.add_argument("--port=19222")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")

In [37]:
driver = uc.Chrome()

WebDriverException: Message: unknown error: cannot connect to chrome at 127.0.0.1:32963
from chrome not reachable
Stacktrace:
#0 0x55e464aa4d93 <unknown>
#1 0x55e464788177 <unknown>
#2 0x55e46477302a <unknown>
#3 0x55e4647c1cf8 <unknown>
#4 0x55e4647b81cb <unknown>
#5 0x55e464802b7c <unknown>
#6 0x55e4647f61e3 <unknown>
#7 0x55e4647c6135 <unknown>
#8 0x55e4647c713e <unknown>
#9 0x55e464a68e4b <unknown>
#10 0x55e464a6cdfa <unknown>
#11 0x55e464a556d5 <unknown>
#12 0x55e464a6da6f <unknown>
#13 0x55e464a3969f <unknown>
#14 0x55e464a92098 <unknown>
#15 0x55e464a92262 <unknown>
#16 0x55e464aa3f34 <unknown>
#17 0x7f8646d36609 start_thread


In [23]:
driver = uc.Chrome(use_subprocess=True)

WebDriverException: Message: unknown error: cannot connect to chrome at 127.0.0.1:56457
from chrome not reachable
Stacktrace:
#0 0x5596e8109d93 <unknown>
#1 0x5596e7ded177 <unknown>
#2 0x5596e7dd802a <unknown>
#3 0x5596e7e26cf8 <unknown>
#4 0x5596e7e1d1cb <unknown>
#5 0x5596e7e67b7c <unknown>
#6 0x5596e7e5b1e3 <unknown>
#7 0x5596e7e2b135 <unknown>
#8 0x5596e7e2c13e <unknown>
#9 0x5596e80cde4b <unknown>
#10 0x5596e80d1dfa <unknown>
#11 0x5596e80ba6d5 <unknown>
#12 0x5596e80d2a6f <unknown>
#13 0x5596e809e69f <unknown>
#14 0x5596e80f7098 <unknown>
#15 0x5596e80f7262 <unknown>
#16 0x5596e8108f34 <unknown>
#17 0x7f243edca609 start_thread


#47565

In [24]:
driver = uc.Chrome(use_subprocess=False)

WebDriverException: Message: unknown error: cannot connect to chrome at 127.0.0.1:54155
from chrome not reachable
Stacktrace:
#0 0x5585d83b9d93 <unknown>
#1 0x5585d809d177 <unknown>
#2 0x5585d808802a <unknown>
#3 0x5585d80d6cf8 <unknown>
#4 0x5585d80cd1cb <unknown>
#5 0x5585d8117b7c <unknown>
#6 0x5585d810b1e3 <unknown>
#7 0x5585d80db135 <unknown>
#8 0x5585d80dc13e <unknown>
#9 0x5585d837de4b <unknown>
#10 0x5585d8381dfa <unknown>
#11 0x5585d836a6d5 <unknown>
#12 0x5585d8382a6f <unknown>
#13 0x5585d834e69f <unknown>
#14 0x5585d83a7098 <unknown>
#15 0x5585d83a7262 <unknown>
#16 0x5585d83b8f34 <unknown>
#17 0x7f7963a55609 start_thread


In [25]:
driver = uc.Chrome(options = options)

WebDriverException: Message: unknown error: cannot connect to chrome at 127.0.0.1:50939
from chrome not reachable
Stacktrace:
#0 0x56357d497d93 <unknown>
#1 0x56357d17b177 <unknown>
#2 0x56357d16602a <unknown>
#3 0x56357d1b4cf8 <unknown>
#4 0x56357d1ab1cb <unknown>
#5 0x56357d1f5b7c <unknown>
#6 0x56357d1e91e3 <unknown>
#7 0x56357d1b9135 <unknown>
#8 0x56357d1ba13e <unknown>
#9 0x56357d45be4b <unknown>
#10 0x56357d45fdfa <unknown>
#11 0x56357d4486d5 <unknown>
#12 0x56357d460a6f <unknown>
#13 0x56357d42c69f <unknown>
#14 0x56357d485098 <unknown>
#15 0x56357d485262 <unknown>
#16 0x56357d496f34 <unknown>
#17 0x7fe6ff55d609 start_thread


In [30]:
driver = uc.Chrome(use_subprocess=True, options = options)

WebDriverException: Message: unknown error: cannot connect to chrome at 127.0.0.1:58171
from chrome not reachable
Stacktrace:
#0 0x557e48362d93 <unknown>
#1 0x557e48046177 <unknown>
#2 0x557e4803102a <unknown>
#3 0x557e4807fcf8 <unknown>
#4 0x557e480761cb <unknown>
#5 0x557e480c0b7c <unknown>
#6 0x557e480b41e3 <unknown>
#7 0x557e48084135 <unknown>
#8 0x557e4808513e <unknown>
#9 0x557e48326e4b <unknown>
#10 0x557e4832adfa <unknown>
#11 0x557e483136d5 <unknown>
#12 0x557e4832ba6f <unknown>
#13 0x557e482f769f <unknown>
#14 0x557e48350098 <unknown>
#15 0x557e48350262 <unknown>
#16 0x557e48361f34 <unknown>
#17 0x7f735bb0d609 start_thread


In [35]:
#127.0.0.1:35573 

In [34]:
driver = uc.Chrome(use_subprocess=False,options = options)

WebDriverException: Message: unknown error: cannot connect to chrome at 127.0.0.1:45717
from chrome not reachable
Stacktrace:
#0 0x564e49a46d93 <unknown>
#1 0x564e4972a177 <unknown>
#2 0x564e4971502a <unknown>
#3 0x564e49763cf8 <unknown>
#4 0x564e4975a1cb <unknown>
#5 0x564e497a4b7c <unknown>
#6 0x564e497981e3 <unknown>
#7 0x564e49768135 <unknown>
#8 0x564e4976913e <unknown>
#9 0x564e49a0ae4b <unknown>
#10 0x564e49a0edfa <unknown>
#11 0x564e499f76d5 <unknown>
#12 0x564e49a0fa6f <unknown>
#13 0x564e499db69f <unknown>
#14 0x564e49a34098 <unknown>
#15 0x564e49a34262 <unknown>
#16 0x564e49a45f34 <unknown>
#17 0x7efe8dd52609 start_thread


In [ ]:
driver.get('http://101katok.ru')